In [1]:
import pandas as pd


In [2]:
df=pd.read_csv("C:\Music Recomender system\spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [5]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [6]:
df.shape

(57650, 4)

In [7]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [8]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [9]:
df.head(10)

,artist,song,text
0,Unkle,Eye For An Eye,Even now in heaven \r\nThere were angels carr...
1,Bette Midler,Friends,And I am all alone. \r\nThere is no one here ...
2,George Jones,Achin' Breakin' Heart,Well I have lived a lot and done my part of ru...
3,Gary Numan,Engineers,All that we are \r\nIs all that we need to be...
4,Underoath,Only Survivor Was Miraculously Unharmed,Someone has sewn me shut \r\nAnd tied me to a...
5,Diana Ross,He's All I Got,You're a girl that hates to see others happy ...
6,Dan Fogelberg,Song For A Carpenter,"Oh, he makes his life as a carpenter \r\nHe w..."
7,Ween,She Fucks Me,(Pork roll egg and cheese on a kaiser bun) \r...
8,R. Kelly,I Like Love,First of all you gotta understand \r\nThis al...
9,Helloween,Come Alive,It don't get better if you're locked behind yo...


In [10]:
df['text'][0]

"Even now in heaven  \r\nThere were angels carrying savage weapons  \r\n  \r\nAn eye for an eye (for an eye)  \r\nA tooth for a tooth (for a tooth)  \r\n  \r\nRun, run, run,  \r\nBut you sure can't hide (hide, hide)  \r\n  \r\nAn eye for an eye (for an eye)  \r\nA tooth for a tooth (for a tooth)  \r\n  \r\nRun, run, run,  \r\nBut you sure can't hide (hide, hide)  \r\n  \r\nIs that room been fit to earth?  \r\nDoesn't help the to grow sunshine?  \r\nIs this darkness all you'll take?  \r\nHave you'd passed through this life?  \r\n  \r\nRun, run, run, but you sure can't hide (hide, hide)  \r\n  \r\nWhere you're going you're not coming back from  \r\n  \r\nRun, run, run, but you sure can't hide (hide, hide)  \r\n  \r\nAn eye for an eye (for an eye)  \r\nA tooth for a tooth (for a tooth)  \r\nRun, run, run,  \r\nBut you sure can't hide (hide, hide)  \r\n  \r\nAn eye for an eye (for an eye)  \r\nA tooth for a tooth (for a tooth)  \r\nRun, run, run,  \r\nBut you sure can't hide (hide, hide)  

In [11]:
df.shape

(5000, 3)

Text Preprocessing

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [14]:
df['text'] = df['text'].apply(lambda x: tokenization(x))
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)
similarity[0]
df[df['song'] == 'Crying Over You']

,artist,song,text


In [15]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [18]:
recommendation('Love')

['Loved',
 'Love To Love You Baby',
 'Give Up',
 "There's Nothing Better Than Love",
 'Crazy Love',
 'I Love You',
 'How Much Love',
 'As Long As You Love Me',
 'Horseshoe Man',
 "I'll Love You For A While",
 'When We Die',
 'In Love With You',
 'Anything',
 "Junior's Wailing",
 'You Are The One',
 'Bad Time',
 'I Love Being Here With You',
 'My Love',
 'But You Know I Love You',
 'Love Me Now']

In [19]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))